In [1]:
#pip install pymysql

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from query_sql import query_stmt
from db_connection import mydb_engine
from sqlalchemy import text
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import get_scorer_names

import tensorflow as tf

In [3]:
stmt = """ select * from eth_price;"""
t = text(stmt)
df = pd.read_sql(t, con = mydb_engine)
df

,id,datetime,open,high,low,close,volume,pct_chg,w_avg_price
0,1,2023-12-07 19:50:06,2265.27,2284.6,2220.0,2250.12,346229.8826,-0.669,2251.618661
1,2,2023-12-07 19:50:07,2265.27,2284.6,2220.0,2250.30,346234.7202,-0.661,2251.618640
2,3,2023-12-07 19:50:08,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
3,4,2023-12-07 19:50:09,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
4,5,2023-12-07 19:50:10,2265.27,2284.6,2220.0,2250.42,346238.9490,-0.656,2251.618625
...,...,...,...,...,...,...,...,...,...
29568,29569,2023-12-08 04:13:37,2259.78,2360.0,2220.0,2354.66,473454.9189,4.199,2285.303926
29569,29570,2023-12-08 04:13:38,2259.78,2360.0,2220.0,2354.66,473455.1169,4.199,2285.303955
29570,29571,2023-12-08 04:13:39,2259.78,2360.0,2220.0,2354.65,473457.0187,4.198,2285.304234
29571,29572,2023-12-08 04:13:40,2259.78,2360.0,2220.0,2354.65,473457.0434,4.198,2285.304237


In [4]:
df = query_stmt(mydb_engine)

Preprocess Dataset

In [5]:
df2 = df.copy() #.set_index('datetime')

In [6]:
df2

,id,datetime,open,high,low,close,volume,pct_chg,w_avg_price
0,1,2023-12-07 19:50:06,2265.27,2284.6,2220.0,2250.12,346229.8826,-0.669,2251.618661
1,2,2023-12-07 19:50:07,2265.27,2284.6,2220.0,2250.30,346234.7202,-0.661,2251.618640
2,3,2023-12-07 19:50:08,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
3,4,2023-12-07 19:50:09,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
4,5,2023-12-07 19:50:10,2265.27,2284.6,2220.0,2250.42,346238.9490,-0.656,2251.618625
...,...,...,...,...,...,...,...,...,...
29568,29569,2023-12-08 04:13:37,2259.78,2360.0,2220.0,2354.66,473454.9189,4.199,2285.303926
29569,29570,2023-12-08 04:13:38,2259.78,2360.0,2220.0,2354.66,473455.1169,4.199,2285.303955
29570,29571,2023-12-08 04:13:39,2259.78,2360.0,2220.0,2354.65,473457.0187,4.198,2285.304234
29571,29572,2023-12-08 04:13:40,2259.78,2360.0,2220.0,2354.65,473457.0434,4.198,2285.304237


In [7]:
# extract date and time
df2['date'] = df2['datetime'].dt.normalize()
df2['hour'] = df2['datetime'].dt.hour
df2['min'] = df2['datetime'].dt.minute

# create Y column
df2['nxt_3m_close'] = df2['close'].shift(-180)
df2['return_chg3m'] = (df2['nxt_3m_close'] - df2['close']) / df2['close']

# drop NA
df2 = df2.loc[df2['return_chg3m'].isna() == False]
df2.head()

,id,datetime,open,high,low,close,volume,pct_chg,w_avg_price,date,hour,min,nxt_3m_close,return_chg3m
0,1,2023-12-07 19:50:06,2265.27,2284.6,2220.0,2250.12,346229.8826,-0.669,2251.618661,2023-12-07,19,50,2250.87,0.000333
1,2,2023-12-07 19:50:07,2265.27,2284.6,2220.0,2250.30,346234.7202,-0.661,2251.618640,2023-12-07,19,50,2250.87,0.000253
2,3,2023-12-07 19:50:08,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625,2023-12-07,19,50,2250.87,0.000200
3,4,2023-12-07 19:50:09,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625,2023-12-07,19,50,2250.87,0.000200
4,5,2023-12-07 19:50:10,2265.27,2284.6,2220.0,2250.42,346238.9490,-0.656,2251.618625,2023-12-07,19,50,2250.87,0.000200


In [8]:
len(df2)

29393

In [9]:
len(df2.loc[df2['return_chg3m'].isna() == False])

29393

In [10]:
len(df2.loc[df2['return_chg3m'].isna()])

0

In [11]:
print(df2.columns)

Index(['id', 'datetime', 'open', 'high', 'low', 'close', 'volume', 'pct_chg',
       'w_avg_price', 'date', 'hour', 'min', 'nxt_3m_close', 'return_chg3m'],
      dtype='object')


Random Forest Model

In [12]:
#column_to_drop = ['id', 'datetime', 'volume', 'pct_chg', 'w_avg_price', 'date', 'hour', 'min', 'nxt_3m_close', 'return_chg3m']
#X = df2.drop(columns=column_to_drop, inplace=True)

columns_to_include = ['open', 'high', 'low', 'close']
X = df2[columns_to_include]

y = df2['nxt_3m_close']

In [13]:
X

,open,high,low,close
0,2265.27,2284.6,2220.0,2250.12
1,2265.27,2284.6,2220.0,2250.30
2,2265.27,2284.6,2220.0,2250.42
3,2265.27,2284.6,2220.0,2250.42
4,2265.27,2284.6,2220.0,2250.42
...,...,...,...,...
29388,2258.01,2356.4,2220.0,2356.01
29389,2258.01,2356.4,2220.0,2356.15
29390,2258.01,2356.4,2220.0,2356.20
29391,2258.01,2356.4,2220.0,2356.39


In [14]:
y

0        2250.87
1        2250.87
2        2250.87
3        2250.87
4        2250.87
          ...   
29388    2354.66
29389    2354.66
29390    2354.65
29391    2354.65
29392    2354.65
Name: nxt_3m_close, Length: 29393, dtype: float64

Train Test Validation Split (80-10-10 ratio with sequence data)

In [15]:
n_data = y.shape[0]
n_train = int( n_data * 0.8 )         # 80% in train set
n_test = int( n_data * 0.1 )          # 10% in test set
n_val = n_data - n_train - n_test     # 10% in validation set

X_train , y_train = X[:n_train] , y[:n_train]
X_val , y_val = X[n_train:n_train+n_val] , y[n_train:n_train+n_val]
X_test , y_test = X[n_train+n_val:] , y[n_train+n_val:]

print("\n=== Train data ===")
print( "#data =", X_train.shape[0] )
print( f"x_train.shape={X_train.shape} , y_train={y_train.shape}" )

print("\n=== Validation data ===")
print( "#data =", X_val.shape[0] )
print( f"x_val.shape={X_val.shape} , y_val.shape={y_val.shape}" )

print("\n=== Test data ===")
print( "#data =", X_test.shape[0] )
print( f"x_test.shape={X_test.shape} , y_test.shape={y_test.shape}" )


=== Train data ===
#data = 23514
x_train.shape=(23514, 4) , y_train=(23514,)

=== Validation data ===
#data = 2940
x_val.shape=(2940, 4) , y_val.shape=(2940,)

=== Test data ===
#data = 2939
x_test.shape=(2939, 4) , y_test.shape=(2939,)


In [16]:
X_train

,open,high,low,close
0,2265.27,2284.6,2220.0,2250.12
1,2265.27,2284.6,2220.0,2250.30
2,2265.27,2284.6,2220.0,2250.42
3,2265.27,2284.6,2220.0,2250.42
4,2265.27,2284.6,2220.0,2250.42
...,...,...,...,...
23509,2267.07,2356.0,2220.0,2330.22
23510,2267.07,2356.0,2220.0,2330.23
23511,2267.07,2356.0,2220.0,2330.23
23512,2267.07,2356.0,2220.0,2330.23


In [17]:
X_test

,open,high,low,close
26454,2261.21,2356.0,2220.0,2346.99
26455,2261.21,2356.0,2220.0,2347.00
26456,2261.21,2356.0,2220.0,2347.00
26457,2261.21,2356.0,2220.0,2346.99
26458,2261.21,2356.0,2220.0,2346.25
...,...,...,...,...
29388,2258.01,2356.4,2220.0,2356.01
29389,2258.01,2356.4,2220.0,2356.15
29390,2258.01,2356.4,2220.0,2356.20
29391,2258.01,2356.4,2220.0,2356.39


In [18]:
X_val

,open,high,low,close
23514,2267.07,2356.0,2220.0,2330.27
23515,2267.07,2356.0,2220.0,2330.27
23516,2267.07,2356.0,2220.0,2330.27
23517,2267.07,2356.0,2220.0,2330.27
23518,2267.07,2356.0,2220.0,2330.28
...,...,...,...,...
26449,2261.21,2356.0,2220.0,2346.93
26450,2261.21,2356.0,2220.0,2346.99
26451,2261.21,2356.0,2220.0,2346.99
26452,2261.21,2356.0,2220.0,2347.00


In [19]:
y_train

0        2250.87
1        2250.87
2        2250.87
3        2250.87
4        2250.87
          ...   
23509    2333.09
23510    2332.89
23511    2332.87
23512    2332.87
23513    2332.84
Name: nxt_3m_close, Length: 23514, dtype: float64

In [20]:
y_test

26454    2342.75
26455    2342.75
26456    2342.75
26457    2342.74
26458    2342.74
          ...   
29388    2354.66
29389    2354.66
29390    2354.65
29391    2354.65
29392    2354.65
Name: nxt_3m_close, Length: 2939, dtype: float64

In [21]:
y_val

23514    2332.81
23515    2332.73
23516    2332.73
23517    2332.73
23518    2332.73
          ...   
26449    2342.67
26450    2342.67
26451    2342.71
26452    2342.71
26453    2342.72
Name: nxt_3m_close, Length: 2940, dtype: float64

In [22]:
# Check valid score names
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weight

GridSearch

In [23]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5)

In [24]:
# Fit the model to the data
grid_search.fit(X_train, y_train)

c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_root_mean_squared_error')

Get the best parameters and best model

In [25]:
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

best_model = grid_search.best_estimator_

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}


Evaluate the model on test dataset

In [26]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

In [27]:
print(f'Mean Squared Error on Test Set: {mse}')

Mean Squared Error on Test Set: 21.456134379197803


In [28]:
# Calculate root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 4.632076681057623
